In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

ml_df = yf.download('^GSPC', start="1978-01-01", interval="1d")

for i in [1, 2, 3, 5, 7, 14, 21]:
    ml_df[f'Close_{i}_Value'] = ml_df['Adj Close'].pct_change(i)
    ml_df[f'Volume_{i}_Value'] = ml_df['Volume'].pct_change(i)
ml_df.dropna(inplace=True)
for i in [3, 7, 14, 21]:
    ml_df[f'Volt_{i}_Value'] = np.log(1 + ml_df['Close_1_Value']).rolling(i).std()
ml_df.dropna(inplace=True)  

sp500_regimes = pd.read_csv('../data/sp500_regimes.csv')
sp500_regimes.set_index('Date', inplace=True)
sp500_regimes.index = pd.to_datetime(sp500_regimes.index)
ml_df.index = pd.to_datetime(ml_df.index)
data = pd.merge(ml_df.drop(columns=['High','Low','Open','Close','Adj Close']), sp500_regimes, on='Date',how='inner')
data['color'] = data['color'].shift(-1)
data.dropna(inplace=True)

/Users/alden/quantfin/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


In [2]:
X = data.drop(columns=['color'])
y = data['color']

In [3]:
X.head()
X_columns = X.columns

In [4]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

# Assume `data` is your DataFrame
scaler = StandardScaler()

# Fit on your data and then transform it
X = scaler.fit_transform(X)

# If you want to convert it back to a DataFrame
X = pd.DataFrame(X, columns=X_columns)

# Assume y is your label data
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y

array([0, 0, 0, ..., 0, 0, 0])

In [5]:
# Assume X and y are your data and labels
train_size = int(len(X) * 0.8)

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]




In [6]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN

# Assuming X is your dataset and y is the target variable
oversampler = SMOTE()  # Choose your oversampler: RandomOverSampler, SMOTE, or ADASYN
X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)

In [7]:
n_timesteps = 1


# Assuming you have X_resampled_reshaped and y_train as your training data
train_size = int(len(X_resampled) * 0.8)  # 80% for training
X_train, X_val = X_resampled[:train_size], X_resampled[train_size:]
y_train, y_val = y_resampled[:train_size], y_resampled[train_size:]

X_train= np.reshape(X_train,(X_train.shape[0], n_timesteps, X_train.shape[1]//n_timesteps))
X_val = np.reshape(X_val,(X_val.shape[0], n_timesteps, X_val.shape[1]//n_timesteps))
X_test_reshaped = np.reshape(X_test,(X_test.shape[0], n_timesteps, X_test.shape[1]//n_timesteps))

In [8]:
X_train.shape

(19897, 1, 28)

In [20]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2
from keras.layers import GaussianNoise

# Create a Sequential model
model = Sequential()
neurons = 587.8010199520079
model.add(LSTM(64, activation='leaky_relu',input_shape=(1, X_train.shape[2]), return_sequences=True,kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001), bias_regularizer=l2(0.001)))
model.add(GaussianNoise(0.01))
model.add(Dropout(0.1))
model.add(LSTM(128, return_sequences=True, activation='leaky_relu'))
# model.add(Dropout(0.3))
model.add(LSTM(128, return_sequences=True, activation='leaky_relu'))

model.add(LSTM(128, return_sequences=True, activation='leaky_relu'))
# # model.add(Dropout(0.3))
# model.add(LSTM(16, return_sequences=True, activation='leaky_relu'))
# model.add(LSTM(256, return_sequences=True, activation='leaky_relu'))
# model.add(Dropout(0.4))
# model.add(LSTM(256, return_sequences=True, activation='leaky_relu'))
# model.add(Dropout(0.4))
# model.add(LSTM(16, return_sequences=True, activation='leaky_relu'))
# model = Sequential()
# model.add(LSTM(int(neurons), input_shape=(1, X_train.shape[2]), return_sequences=True, activation='leaky_relu'))
# model.add(LSTM(int(neurons)//2, return_sequences=True, activation='leaky_relu'))
# model.add(LSTM(int(neurons)//4, return_sequences=True, activation='leaky_relu'))
# model.add(LSTM(int(neurons)//8, return_sequences=True, activation='leaky_relu'))
# model.add(LSTM(int(neurons)//16, activation='leaky_relu'))
# model.add(Dense(4, activation='softmax'))
model.add(LSTM(256, activation='leaky_relu',kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001), bias_regularizer=l2(0.001)))
model.add(Dense(4, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.008326932120119833), metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, epochs=100, batch_size=100,validation_data=(X_val, y_val),callbacks=[early_stopping])

Epoch 1/100
199/199 [==============================] - 5s 12ms/step - loss: 0.7682 - accuracy: 0.7175 - val_loss: 1.3593 - val_accuracy: 0.3112
Epoch 2/100
199/199 [==============================] - 3s 13ms/step - loss: 0.4686 - accuracy: 0.8241 - val_loss: 1.8337 - val_accuracy: 0.0454
Epoch 3/100
199/199 [==============================] - 3s 14ms/step - loss: 0.4984 - accuracy: 0.8273 - val_loss: 1.9018 - val_accuracy: 0.0000e+00
Epoch 4/100
199/199 [==============================] - 2s 12ms/step - loss: 0.5199 - accuracy: 0.8210 - val_loss: 1.1119 - val_accuracy: 0.3819
Epoch 5/100
199/199 [==============================] - 2s 12ms/step - loss: 0.4400 - accuracy: 0.8489 - val_loss: 0.8946 - val_accuracy: 0.5250
Epoch 6/100
199/199 [==============================] - 2s 12ms/step - loss: 0.4360 - accuracy: 0.8527 - val_loss: 0.8309 - val_accuracy: 0.5793
Epoch 7/100
199/199 [==============================] - 2s 11ms/step - loss: 0.4216 - accuracy: 0.8722 - val_loss: 0.9141 - val_accur

In [21]:
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test, verbose=0)

print(f'Test accuracy: {test_accuracy*100}%')

Test accuracy: 91.87295436859131%


In [16]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from bayes_opt import BayesianOptimization
 
# Define a function to build and compile the model

def create_model(lr, neurons):
    model = Sequential()
    model.add(LSTM(int(neurons), input_shape=(1, X_train.shape[2]), return_sequences=True, activation='leaky_relu'))
    model.add(LSTM(int(neurons)//2, return_sequences=True, activation='leaky_relu'))
    model.add(LSTM(int(neurons)//4, return_sequences=True, activation='leaky_relu'))
    model.add(LSTM(int(neurons)//8, return_sequences=True, activation='leaky_relu'))
    model.add(LSTM(int(neurons)//16, activation='leaky_relu'))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=lr), metrics=['accuracy'])
    return model

# Define a function for the hyperparameter optimization
def optimize_model(lr, neurons):
    model = create_model(lr, neurons)
    # Assuming you have X_train, y_train as your training data
    model.fit(X_train, y_train, epochs=5, batch_size=128, verbose=0)
    # Assuming you have X_val, y_val as your validation data
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    return val_accuracy

# Define the bounds of the hyperparameters to optimize
bounds = {'lr': (0.0001, 0.01), 'neurons': (128, 1024)}

optimizer = BayesianOptimization(
    f=optimize_model,
    pbounds=bounds,
    random_state=1,
)

optimizer.maximize(init_points=10, n_iter=50)

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)

|   iter    |  target   |    lr     |  neurons  |
-------------------------------------------------


| 1         | 0.9851    | 0.004229  | 773.4     |


| 2         | 0.8961    | 0.0001011 | 398.9     |
| 3         | 0.5212    | 0.001553  | 210.7     |


| 4         | 0.9765    | 0.001944  | 437.6     |


| 5         | 0.8959    | 0.004028  | 610.8     |


...

| 56        | 0.9594    | 0.01      | 494.7     |


| 57        | 0.956     | 0.006691  | 752.9     |


| 58        | 0.9102    | 0.007406  | 960.3     |


| 59        | 0.9672    | 0.0009342 | 693.3     |


| 60        | 0.9087    | 0.002737  | 895.5     |
Iteration 0: 
	{'target': 0.9851256012916565, 'params': {'lr': 0.004228517846555483, 'neurons': 773.4107461241737}}
Iteration 1: 
	{'target': 0.8960803747177124, 'params': {'lr': 0.00010113231069171439, 'neurons': 398.88998507812846}}
Iteration 2: 
	{'target': 0.5212060213088989, 'params': {'lr': 0.0015528833190894193, 'neurons': 210.73538091284283}}
Iteration 3: 
	{'target': 0.9764823913574219, 'params': {'lr': 0.0019439760926389421, 'neurons': 437.62241143057076}}
Iteration 4: 
	{'target': 0.8958793878555298, 'params': {'lr': 0.004027997994883633, 'neurons': 610.7797936670079}}
Iteration 5: 
	{'target': 0.9443216323852539, 'params': {'lr': 0.004250025692592619, 'neurons': 741.9566723554965}}
Iteration 6: 
	{'target': 0.979497492313385, 'params': {'lr': 0.0021240772723420225, 'neurons': 914.7932230062871}}
Iteration 7: 
	{'target': 0.9636180996894836, 'params': {'lr': 0.00037113717265946903, 'neurons': 728.7388891198484}}
Iteration 8: 

In [22]:
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred, axis=-1)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
f1 = f1_score(y_test, y_pred, average='weighted')  # Use 'weighted' if you have a multi-class problem
print(f'F1 Score: {f1}')
y_prob_pred = model.predict(X_test_reshaped)
acc_score = accuracy_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob_pred, multi_class='ovr')
print(f'AUC-ROC (One-vs-Rest): {auc_roc}')


67/67 [==============================] - 0s 2ms/step
Accuracy: 91.87%
F1 Score: 0.9072968676964156
67/67 [==============================] - 0s 2ms/step
AUC-ROC (One-vs-Rest): 0.9733614320055951
